# Import libraries

In [ ]:
import requests as req
import pandas as pd
import os
from functools import reduce
import json
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

# Authenticate to API

In [ ]:
from configparser import ConfigParser
config = ConfigParser()
config.read("/Users/nathansuberi/Desktop/WRI_Programming/cred/.env")
api_token = config.get("auth", "rw_api_token")

AUTH_TOKEN = api_token # <Insert Auth Token Here>

# Download upload config

In [ ]:
#### Download Google Spreadsheets ####
# ImageCollection Upload Config
!curl "https://docs.google.com/spreadsheets/d/1NCL998eESaXcodvzTE7e9_zpBxVAG4HiQXpzP1E_Cpk/export?format=tsv" > ic_upload_config.tsv
ic_upload_config = pd.read_csv(open("ic_upload_config.tsv", "r"), sep="\t", index_col=None)
os.remove("ic_upload_config.tsv")

In [ ]:
ic_upload_config

# Helper functions

In [ ]:
def createHeaders():
    return {
        'content-type': "application/json",
        'authorization': "Bearer {}".format( AUTH_TOKEN )
    }

def upload_ic_to_backoffice(wri_id, imageCollectionName, datasetName):
    
    ds_specs = {
        "connectorType":"rest",
        "provider":"gee",
        "tableName":imageCollectionName,
        "application":["rw"],
        "geoInfo":True,
        "type":"raster",
        "name":"{}_{}".format(wri_id, datasetName)
    }

    create_res = req.request("POST", 
                      'https://staging-api.globalforestwatch.org/v1/dataset', 
                      data=json.dumps(ds_specs), 
                      headers = createHeaders())

    logging.info(create_res.text)

    return create_res.json()['data']['id']


def autogen_layer_def(imageCollectionName, layerName, sld):  
    
    layer_template = {
        "name":layerName,
        "application":["rw"],"iso":[],
        "default":True,
        "protected":False,
        "layerConfig":{
            "type":"gee",
            "position":"last",
            "isImageCollection":True,
            #"filterDates":["2010-01-01","2017-10-24"],
            "assetId":imageCollectionName,
            "body":{
                "styleType":"sld",
                "sldValue":sld
            }},
        "legendConfig":{},
        "interactionConfig":{},
        "applicationConfig":{},
        "staticImageConfig":{},
    }
    
    #layer_template['layerConfig']['timeline'] = True
    #layer_template['layerConfig']['order'] = int(str_year)
    #layer_template['layerConfig']['timelineLabel'] = str(str_year)
   
    return layer_template
        
def upload_layer_def_to_backoffice(layer_def, rw_id):
    url = "https://staging-api.globalforestwatch.org/v1/dataset/{}/layer".format(rw_id)
    res = req.request("POST", url, data=json.dumps(layer_def), headers = createHeaders())
    return res.text

def prepare_data(obj, info):
    wri_id = info[0]
    imageCollectionName = info[1]
    datasetName = info[2]
    sld = info[3]
        
    rw_id = upload_ic_to_backoffice(wri_id, imageCollectionName, datasetName)
    layer_def = autogen_layer_def(imageCollectionName, 'Most recent ' + datasetName, sld)
    layer_on_backoffice = upload_layer_def_to_backoffice(layer_def, rw_id)
    
    #layer_defs = list(map(lambda year: autogen_layer_def(year, max_year, rw_id, carto_name, cartocss, legend, interaction, basesql), wbg_ds_georef['year'].unique()))
    #logging.info(layer_defs)
    #layer_defs_on_backoffice = list(map(lambda ldef: upload_layer_defs_to_backoffice(ldef, rw_id), layer_defs))
    #logging.info(layer_defs_on_backoffice)
    #vocab_on_backoffice = load_tags_to_backoffice(tags, rw_id)
    #logging.info(vocab_on_backoffice)
    
    
    obj[wri_id] = {
        'rw_id':rw_id,
        'layers':json.loads(layer_on_backoffice),
        #'tags':vocab_on_backoffice
    }
    
    return obj

In [ ]:
raster_uploaded = reduce(prepare_data, ic_upload_config.values, {})

In [ ]:
def gen_lookup_links(obj_info):
    layer_id = obj_info['layers']['data']['id']
    url = 'https://staging-api.globalforestwatch.org/layer/{}/tile/gee/'.format(layer_id)
    return url + '{z}/{x}/{y}'

In [ ]:
raster_uploaded

In [ ]:
list(map(gen_lookup_links, raster_uploaded.values()))